In [1]:
# 导入所需的库
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
import os
from sklearn.metrics import mean_squared_error

In [2]:
raw_train_data = pd.read_csv(r"C:\Users\yubin\Desktop\综合课程设计3\锂电池\train_data.csv")
raw_test_data = pd.read_csv(r"C:\Users\yubin\Desktop\综合课程设计3\锂电池\test_data.csv")
raw_dev_data = pd.read_csv(r"C:\Users\yubin\Desktop\综合课程设计3\锂电池\dev_data.csv")

In [15]:
# 删除第一列
train_data = raw_train_data.drop(raw_train_data.columns[0], axis=1)
dev_data = raw_dev_data.drop(raw_dev_data.columns[0], axis=1)
test_data = raw_test_data.drop(raw_test_data.columns[0], axis=1)
t = test_data[test_data['SoH'] != -100]
d = dev_data[dev_data['SoH'] != -100]
combined_df = pd.concat([t, d])
train_data=pd.concat([train_data,combined_df])
# 将值为0的数替换为NaN
train_data.replace(0, np.nan, inplace=True)
test_data.replace(0, np.nan, inplace=True)
train_data=train_data.dropna();
#训练集y
train_y=train_data['SoH']
# #填补缺失值
# train_data['CCCT'] = train_data['CCCT'].fillna(train_data['CCCT'].mean())
# train_data['CVCT'] = train_data['CVCT'].fillna(train_data['CVCT'].mean())
train_data=train_data.drop(columns=['SoH'])
# 对"XX"列进行独热编码
train_data = pd.get_dummies(train_data, columns=["CS_Name"])
# #充电时长
# train_data['CC/CV']=train_data['CCCT']/train_data['CVCT']
# col_to_multiply = {'cycle','CS_Name_CS2_35','CS_Name_CS2_36','CS_Name_CS2_37','CS_Name_CS2_38'}
# # 获取除了 col_to_multiply 列之外的所有列
# cols_to_multiply_with = train_data.drop(col_to_multiply, axis=1).columns
# # 循环相乘
# for col in cols_to_multiply_with:
#     new_col_name = f"cycle_mul_{col}"
#     train_data[new_col_name] = train_data['cycle'] * train_data[col]
#
# train_data['log_cycle'] = np.log(train_data['cycle'])
# train_data['2_cycle'] = train_data['cycle']**2

In [16]:
t_data = test_data[test_data['SoH'] == -100]
#填补缺失值
t_data['CCCT'] = t_data['CCCT'].fillna(t_data['CCCT'].mean())
t_data['CVCT'] = t_data['CVCT'].fillna(t_data['CVCT'].mean())
t_data=t_data.drop(columns=['SoH'])
t_data = pd.get_dummies(t_data, columns=["CS_Name"])
# #充电时长
# t_data['CC/CV']=t_data['CCCT']/t_data['CVCT']
# # 选择要循环相乘的列
# col_to_multiply = {'cycle','CS_Name_CS2_35','CS_Name_CS2_36','CS_Name_CS2_37','CS_Name_CS2_38'}
# # 获取除了 col_to_multiply 列之外的所有列
# cols_to_multiply_with = t_data.drop(col_to_multiply, axis=1).columns
# # 循环相乘
# for col in cols_to_multiply_with:
#     new_col_name = f"cycle_mul_{col}"
#     t_data[new_col_name] = t_data['cycle'] * t_data[col]
# t_data['log_cycle'] = np.log(t_data['cycle'])
# t_data['2_cycle'] = t_data['cycle']**2;

C:\Users\yubin\AppData\Local\Temp\ipykernel_16868\383323821.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_data['CCCT'] = t_data['CCCT'].fillna(t_data['CCCT'].mean())
C:\Users\yubin\AppData\Local\Temp\ipykernel_16868\383323821.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_data['CVCT'] = t_data['CVCT'].fillna(t_data['CVCT'].mean())


In [17]:

# 建立多模型
# LightGBM模型参数
lgb_params = {
    'n_estimators': 500,
    'objective': 'regression',  # 目标函数
    'metric': 'rmse',  # 评估指标
    'learning_rate': 0.03,  # 学习率
    'feature_fraction': 0.8,  # 训练特征采样比例
    'bagging_fraction': 0.8,  # 训练数据采样比例
    'bagging_freq': 5,  # bagging的频率
    'verbose': 0  # 显示训练过程信息的频率
}

# 随机森林模型参数
rf_params = {
    'n_estimators': 800,  # 决策树数量
    'max_depth': None,  # 决策树最大深度，None表示不限制深度
    'max_features': 'auto',  # 寻找最佳分割时考虑的特征数量
    'random_state': 42  # 随机种子，以确保可重复性
}

# XGBoost模型参数
xgb_params = {
    'objective': 'reg:squarederror',  # 目标函数
    'learning_rate': 0.1,  # 学习率
    'max_depth': 7,  # 树的最大深度
    'min_child_weight': 1,  # 叶子节点最小权重
    'n_estimators': 500,  # 决策树数量
    'random_state': 42  # 随机种子，以确保可重复性
}
models = {
    'lgb': lgb.LGBMRegressor(**lgb_params),
    'rf': RandomForestRegressor(**rf_params),
    'xgb': xgb.XGBRegressor(
        n_estimators=1000,  # 迭代次数
        learning_rate=0.01,  # 学习率
        objective='reg:squarederror',  # 损失函数
        random_state=42
    )
}

train_x=train_data;

t_data_x=t_data;


for name, model in models.items():
    # 训练模型
    model.fit(train_x, train_y)

    # 预测
    test_y = model.predict(t_data_x)
# model.fit(train_x, train_y)
# test_y = model.predict(t_data_x)


# rmse = np.sqrt(mean_squared_error(test_y, train_y))
# rmse

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 

In [18]:
# 假设 test_y 是一个一维数组，你可以将其转换为 DataFrame 对象
test_y_df = pd.DataFrame({'result': test_y})
test_=raw_test_data[raw_test_data['SoH'] == -100]
test_ = test_.reset_index(drop=True)
# 按列拼接预测结果和 t_data_x
merged_df = pd.concat([test_, test_y_df],axis=1)
# merged_df.rename(columns={'Unamed: 0': 'first'}, inplace=True)
selected_columns = ['Unnamed: 0', 'cycle', 'CS_Name','result']
# 选择特定列
new_df = merged_df.loc[:, selected_columns]
new_df.to_csv('result.csv', index=False)
# # 保存拼接后的结果到新的 CSV 文件
# merged_df.to_csv('merged_result.csv', index=False)

In [8]:
m_df = pd.concat([train_x, train_y],axis=1)
m_df.to_csv('m.csv', index=False)

In [19]:
new_df

,Unnamed: 0,cycle,CS_Name,result
0,105,735.0,CS2_35,0.477882
1,106,736.0,CS2_35,0.461272
2,107,737.0,CS2_35,0.451485
3,108,738.0,CS2_35,0.453611
4,109,739.0,CS2_35,0.434205
...,...,...,...,...
115,147,857.0,CS2_38,0.424653
116,148,858.0,CS2_38,0.424912
117,149,859.0,CS2_38,0.425265
118,150,860.0,CS2_38,0.425265


In [20]:
test_y_df

,result
0,0.477882
1,0.461272
2,0.451485
3,0.453611
4,0.434205
...,...
115,0.424653
116,0.424912
117,0.425265
118,0.425265
